In [ ]:
import torch
from PINN import PINN
from Net import Net
import numpy as np
import pandas as pd
import numpy as np
import time
import os

# set default type double
default_type = torch.float64
torch.set_default_dtype(default_type)

In [ ]:
data_dir = f"{os.getcwd()}/pinn_test_data"
# load in data
df_wind_ch4 = pd.read_csv(data_dir + "/wind_ch4.csv")
df_true_emission = pd.read_csv(data_dir + "/selected_controll_release.csv")
source_points = np.load(data_dir + "/source_points.npy") # shape=(n_source, 3)
sensor_points = np.load(data_dir + "/sensor_points.npy") # shape=(n_sensor, 3)
#col_points = np.load(data_dir + "/col_points.npy")  # shape=(n_col, 3)
df_bounds = pd.read_csv(data_dir + "/bounds.csv", dtype='float32')
x_min = df_bounds['x_min'][0]
x_max = df_bounds['x_max'][0]
y_min = df_bounds['y_min'][0]
y_max = df_bounds['y_max'][0]
z_min = df_bounds['z_min'][0]
z_max = df_bounds['z_max'][0]


ws = df_wind_ch4['wind_speed.m/s'].to_numpy() # shape=(N_t,)
wd = df_wind_ch4['wind_direction'].to_numpy() # shape=(N_t,)
ch4 = np.transpose(df_wind_ch4.iloc[:, 3:].to_numpy()) # shape=(N_obs, N_t)
sensor_names = df_wind_ch4.columns[3:]

In [ ]:
print(source_points)

In [ ]:
model = PINN([300,300,300])
model.set_location(source_points,[1,x_max,y_max,z_max],source_values=[0,10,0,0,0],sigma = .1)
tfinal = 1.

In [ ]:
optimizer = torch.optim.Adam(model.net.parameters(), lr=1e-3)
# from torch.optim.lr_scheduler import ExponentialLR

# scheduler = ExponentialLR(optimizer, gamma=0.995)  # Decay LR by 5% every epoch

In [ ]:
n= int(1e2)
sn = 3000
best_loss = np.inf
print_freq = 1
max_epochs = int(5e3)
sample_freq = 10
n_source_samples = int(1e3)

standard_dev = 1e-1

for epoch in range(max_epochs):
    start_time = time.time()
    if epoch % sample_freq == 0:
        source_collocation_points = model.source_points(n_source_samples,standard_dev)
        ic_col = torch.cat([torch.rand(sn,1)*tfinal*.1, torch.rand(sn,1)*x_max, torch.rand(sn,1)*y_max, torch.rand(sn,1)*z_max], dim=1)

        collocation_points = torch.cat([torch.rand(n,1)*tfinal, torch.rand(n,1)*x_max, torch.rand(n,1)*y_max, torch.rand(n,1)*z_max], dim=1)
        collocation_points = torch.empty(0,4)
        # ic_col = torch.empty(0,4)
        ic_col.requires_grad_()
        collocation_points = torch.cat([collocation_points,ic_col,source_collocation_points])
        wind_vector = torch.ones(len(collocation_points),2)*10.0
    optimizer.zero_grad()

    loss_1 ,pde_1 = model.loss_function(collocation_points,wind_vector) # PDE residual loss
    # loss_2,pde_2 = model.loss_function(source_collocation_points, uv_points, source_term = source_values) # source term PDE residual loss 
    # loss_3 ,pde_3 = model.loss_function(torch.concat([collocation_points,source_collocation_points]),torch.concat([collocation_points,source_collocation_points]))

    loss = loss_1 
    
    loss.backward()

    # compute norm of gradient of the network
    grad_norm = 0
    for p in model.net.parameters():
        # if p.grad.data is not None:
        if p.grad is not None:
            grad_norm += p.grad.data.norm().item()**2
    grad_norm = grad_norm**0.5


    if loss.item() < best_loss:
        torch.save(model,'best_mod.m')
    optimizer.step()

    end_time = time.time()
    epoch_time = end_time - start_time
    # scheduler.step()

    if epoch % print_freq == 0:

        # print epoch and loss using %1.3e format
        print('epoch: %d, loss: %1.3e, grad_norm: %1.3e, pde_res: %1.3e, time: %1.3e' % (epoch, loss.item(), grad_norm, loss_1.item(), epoch_time))



In [ ]:
source_collocation_points

In [ ]:
model = torch.load('best_mod.pth')

In [ ]:
import matplotlib.pyplot as plt

# load the best model 
Z_value = 2
# net.load_state_dict(torch.load('best_model.pth'))

# Define the grid and time steps
n= 500
x_grid = np.linspace(0, 2.0*x_max, n)
y_grid = np.linspace(0, 2.0*y_max, n)
z_grid = np.linspace(0, 2, n)

X, Y, = np.meshgrid(x_grid, y_grid)
Z= X * 0 + Z_value

grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
# grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T  # Flatten the grid
grid_points = torch.tensor(grid_points, dtype=torch.float32)

time_steps = np.linspace(0, tfinal, 50)  # 10 time steps from 0 to 1


'''
# Plot the concentration over time
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10)  # Plot concentration as a contour plot
    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # fix colorbar from 0 to 1 
    # plt.clim(0, 10.0)  # Set colorbar limits
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    # fix colorbar 
    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
'''

In [ ]:
# save as a GIF
import imageio
import os

images = []
# source_loc = torch.tensor([[.5,.5,.5]])
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=default_type)  # Time input
    concentration = model.forward(torch.cat([t_tensor,grid_points],dim=1),scaled=False).cpu().detach().numpy().reshape(n, n)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=50)  # Plot concentration as a contour plot
    plt.plot(model.source_locs[1,0], model.source_locs[1,1], 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.title(f"t = {t:.2f}")

    plt.xlabel('x')
    plt.ylabel('y')
    plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
    plt.close()
    
    images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list
    os.remove(f"concentration_t_{t:.2f}.png")  # Remove the image file

imageio.mimsave(f'test_visual.gif', images)  # Save the images as a GIF



In [ ]:
model.source_mixture
source_term = torch.tensor(np.exp(model.source_mixture.score_samples(model.source_locs)))
source_term

In [ ]:
print(len(model.net.hidden))

In [ ]:
print(model.source_mixture.weights_)

In [ ]:
tfinal